<a href="https://colab.research.google.com/github/MIAShield/MIAShield/blob/main/Evaluation_of_MIAShield_(Cifar10_and_100).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install all required library
!sudo pip install Pillow
!sudo pip install imagehash
!pip install tensorflow_privacy

In [ ]:
!pip install torch torchvision
!pip install sklearn

In [ ]:
#!pip install opencv-python-headless==4.5.4.60
#!pip install fiftyone --no-binary fiftyone,voxel51-eta

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import glob
import itertools
import collections
#import fiftyone as fo
#import fiftyone.zoo as foz

from PIL import Image
import cv2
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np
import torch
import imagehash
import hashlib


#importing libraries
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import sys
from matplotlib import pyplot
from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from tensorflow.keras.optimizers import SGD
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# general imports
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from scipy import special

# tensorflow imports
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, MaxPooling2D,Dropout, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score

In [ ]:
# tensorflow-privacy libraries
from tensorflow_privacy.privacy.membership_inference_attack import membership_inference_attack as mia
from tensorflow_privacy.privacy.membership_inference_attack.data_structures import AttackInputData
from tensorflow_privacy.privacy.membership_inference_attack.data_structures import SlicingSpec
from tensorflow_privacy.privacy.membership_inference_attack.data_structures import AttackType

/usr/local/lib/python3.7/dist-packages/tensorflow_privacy/privacy/membership_inference_attack/__init__.py:19: UserWarning: 
Membership inference attack sources were moved. Please replace
import tensorflow_privacy.privacy.membership_inference_attack

with
import tensorflow_privacy.privacy.privacy_tests.membership_inference_attack
  "\nMembership inference attack sources were moved. Please replace"


Initialization

In [ ]:
dataset='Cifar10'            #dataset, 'Cifar10'/'Cifar100'
EO='EO2'                     #EO: 'EO2','EO3', 'EO4', 'EO5'
batch_size=128

In [ ]:
if dataset=='Cifar10':
  #define all the variables
#cifar10
  shape = (32, 32, 3)
  num_class=10
  learning_rate=.01
  epochs=60
  M=5
elif dataset=='Cifar100':
  #cifar100
  shape = (32, 32, 3)
  num_class=100
  learning_rate=.01
  epochs=130
  M=4


In [ ]:
def load_cifar10():
  """Loads cifar10-Dataset and preprocesses to combine training and test data."""
  
  # load the existing CIFAR10 dataset that comes in form of traing + test data and labels
  train, test = tf.keras.datasets.cifar10.load_data()
  train_data, train_labels = train
  test_data, test_labels = test

  # scale the images from color values 0-255 to numbers from 0-1 to help the training process
  train_data1 = np.array(train_data, dtype=np.float32) / 255
  test_data1 = np.array(test_data, dtype=np.float32) / 255

  # cifar10 labels come one-hot encoded, there
  train_labels = train_labels.flatten()
  test_labels = test_labels.flatten()

  return train_data1, train_labels, test_data1, test_labels, train_data, train_labels, test_data, test_labels 

In [ ]:
def load_cifar100():
  """Loads cifar100-Dataset and preprocesses to combine training and test data."""
  
  # load the existing CIFAR10 dataset that comes in form of traing + test data and labels
  train, test = tf.keras.datasets.cifar100.load_data()
  train_data, train_labels = train
  test_data, test_labels = test

  # scale the images from color values 0-255 to numbers from 0-1 to help the training process
  train_data1 = np.array(train_data, dtype=np.float32) / 255
  test_data1 = np.array(test_data, dtype=np.float32) / 255

  # cifar10 labels come one-hot encoded, there
  train_labels = train_labels.flatten()
  test_labels = test_labels.flatten()

  return train_data1, train_labels, test_data1, test_labels, train_data, train_labels, test_data, test_labels 

In [ ]:
if dataset=='Cifar10':
       train_data, train_labels, test_data, test_labels,trainX1,trainY1, testX1,testY1 = load_cifar10()  
elif dataset=='Cifar100':
       train_data, train_labels, test_data, test_labels,trainX1,trainY1, testX1,testY1 = load_cifar100() 


170508288/170498071 [==============================] - 11s 0us/step


In [ ]:
train_y=to_categorical(train_labels)
test_y=to_categorical(test_labels)

In [ ]:
def AlexnetModel(input_shape,num_classes):
    model = Sequential()
# model.add(Conv2D(96, (11,11), strides=(4,4), activation='relu', padding='same', input_shape=(img_height, img_width, channel,)))
# for original Alexnet
    model.add(Conv2D(48, (3,3), strides=(2,2), activation='relu', padding='same', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
# Local Response normalization for Original Alexnet
    model.add(BatchNormalization())

    model.add(Conv2D(96, (3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
# Local Response normalization for Original Alexnet
    model.add(BatchNormalization())

    model.add(Conv2D(192, (3,3), activation='relu', padding='same'))
    model.add(Conv2D(192, (3,3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
# Local Response normalization for Original Alexnet
    model.add(BatchNormalization())

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model

Common Functions

In [ ]:
def confidence(my_list):
  import numpy as np
  my_array=np.array(my_list)
  p=np.zeros(my_array.shape)
  b=my_array.max(-1)
  condition = my_array == b[..., np.newaxis]
  c = np.where(condition, 1, 0)
  final=np.multiply(c, my_array)
  #my_sum=np.sum(final,axis=0)
  labels=np.argmax(final, axis=1)
  return final

In [ ]:
def label(my_list):
  import numpy as np
  my_array=np.array(my_list)
  p=np.zeros(my_array.shape)
  b=my_array.max(-1)
  condition = my_array == b[..., np.newaxis]
  c = np.where(condition, 1, 0)
  final=np.multiply(c, my_array)
  #my_sum=np.sum(final,axis=0)
  labels=np.argmax(final, axis=1)
  return labels

Non-Private Model_training

In [ ]:
# make the neural network model with the function specified above.
# one model is supposed to train for 10, one for 50 epochs
model_np = AlexnetModel(shape,num_class)
model_np.summary()

In [ ]:
# specify parameters
#learning_rate=.005
optimizer = tf.keras.optimizers.SGD(learning_rate=.01)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

# compile the model
model_np.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
# train the model
history = model_np.fit(train_data, train_y,
                       validation_data=(test_data, test_y),
                       batch_size=128, 
                       epochs=epochs)

In [ ]:
# since we have not specified an activation function on the last layer
# calling the predict function returns the logits
print('Predict on train...')
prob_train = model_np.predict(train_data[0:5000])
print('Predict on test...')
prob_test = model_np.predict(test_data[5000:10000])

Predict on train...
Predict on test...


In [ ]:
train=train_data[0:5000]
trainX=trainX1[0:5000]
test=test_data[5000:10000]
testX=testX1[5000:10000]

In [ ]:
stop!!

Training MIAShield Models 

In [ ]:
#split datasets
#from imblearn.over_sampling import RandomOverSampler

#ros = RandomOverSampler()

#X_ros, y_ros = ros.fit_resample(x_train_split0, trainy_split0)
# split x_train to 5 disjoint datasets
#store each dataset variable name is the list Xtrain
from sklearn.model_selection import StratifiedKFold
X=train_data
Y=train_labels
for i in range(0,M):
  K=M-i
  if(K>=2):
    train=X
    test=Y
    skf = StratifiedKFold(n_splits=K)
    skf.get_n_splits(X, Y)
    #print(skf)
    for train_index, test_index in skf.split(X, Y):
       #print(X.shape)
       X,globals()['x_train_split%s' % i]=train[train_index], train[test_index]
       Y,globals()['trainy_split%s' % i]=test[train_index], test[test_index]
       y_hot=globals()['trainy_split%s' % i]
       y_hot=to_categorical(y_hot)
       globals()['y_train_split%s' % i]=y_hot
       #X=train
       #Y=test
  elif K==1:
    globals()['x_train_split%s' % i]=X
    globals()['trainy_split%s' % i]=Y
    y_hot=to_categorical(Y)
    globals()['y_train_split%s' % i]=y_hot


In [ ]:
#create datagenerator for augmented training
from keras.preprocessing.image import ImageDataGenerator
width_shift = 3/32
height_shift = 3/32
flip = True
rotation=10
datagen = ImageDataGenerator(
    horizontal_flip=flip,
    width_shift_range=.1,
    height_shift_range=.1,
    #brightness_range=[0.02,.1],
    rotation_range=rotation,
    #zoom_range=[0.5,1.0]
    shear_range=0.2,
    zoom_range=0.2,
    )

In [ ]:
for iter in range(0,M):
  modelt=AlexnetModel(shape,num_class)
  optimizer = tf.keras.optimizers.SGD(learning_rate=.01)
  loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
# compile the model
  modelt.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
  #create augmented data
  data_t=globals()['x_train_split%s' % iter]
  target_t=globals()['y_train_split%s' % iter]
  dataGen=datagen
  dataGen.fit(data_t)
  #train=datagen.flow(data_t, target_t, batch_size=128)
# Fit the model
  globals()['history_%s' % iter] = modelt.fit_generator(dataGen.flow(data_t, target_t, batch_size=128, shuffle=False),
                    steps_per_epoch = 10000 / 128, epochs=epochs+30, validation_data=(test_data, test_y))
  globals()['model_%s' % iter]=modelt

In [ ]:
#calculate accuracy/utility test acc
prb=model_0.predict(test_data[0:5000])
test_labeld_result=np.argmax(prb,axis=1)
accuracy_score(test_labels[0:5000],test_labeld_result)
#prob_testd

0.5842

In [ ]:
#from imblearn.over_sampling import RandomOverSampler

#ros = RandomOverSampler()

#X_ros, y_ros = ros.fit_resample(x_train_split0, trainy_split0)
# split x_train to 5 disjoint datasets
#store each dataset variable name is the list Xtrain
from sklearn.model_selection import StratifiedKFold
X=trainX1
Y=trainY1
for i in range(0,M):
  K=M-i
  if(K>=2):
    train=X
    test=Y
    skf = StratifiedKFold(n_splits=K)
    print(skf.get_n_splits(X, Y))
    for train_index, test_index in skf.split(X, Y):
       X,globals()['x_traind%s' % i]=train[train_index], train[test_index]
    #print(X.shape)
       Y,globals()['y_traind%s' % i]=test[train_index], test[test_index]
  elif K==1:
    globals()['x_traind%s' % i]=X
    globals()['y_traind%s' % i]=Y

5
4
3
2


In [ ]:
#target model index
in1=np.zeros(5000)
in2=M*np.ones(5000)
index_target=np.hstack((in1,in2))
index_target

array([0., 0., 0., ..., 5., 5., 5.])

In [ ]:
stop!!!

Exclusion Oracle-2: Hash

In [ ]:
def convert_hash(trainX1):
  lenth=len(trainX1)
  hashid=np.zeros(lenth)
  hash=[]
  for id in range(lenth):
    hashid[id]=id
    #covert in hashes
    #im_array=Image.fromarray(trainX1[id].astype('uint8'), 'RGB')
    im_array=trainX1[id]
    hash1 = int(hashlib.sha1(bytearray(im_array)).hexdigest(), 16) % (10 ** 8)
    #hash1=np.array(hash1)
    #print(type(hash1))
    #print(np.array(hash1))
    hash.append(hash1)
  return np.array(hash)

In [ ]:
#ImageHash
def convert_hash(trainX1):
  lenth=len(trainX1)
  hashid=np.zeros(lenth)
  hash=[]
  for id in range(lenth):
    hashid[id]=id
    #covert in hashes
    im_array=Image.fromarray(trainX1[id].astype('uint8'), 'RGB')
    #im_array=trainX1[id]
    hash1 = imagehash.phash(im_array)
    #print(type(hash1))
    hash1=str(hash1)
    hash1=int(hash1,16)
    #print(type(hash1))
    #print(np.array(hash1))
    hash.append(hash1)
  return np.array(hash)

In [ ]:
def BinarySearch(lys, val):
    first = 0
    last = len(lys)-1
    index = -1
    while (first <= last) and (index == -1):
        mid = (first+last)//2
        if lys[mid] == val:
            index = mid
        else:
            if val<lys[mid]:
                last = mid -1
            else:
                first = mid +1
    return index

In [ ]:
#calculate and store hashes for M=5 #different model and their training dataset
for x in range(0,M):
             #print(j,k+j)
             x1=globals()['x_traind%s' % x]
             x2=convert_hash(x1)
             globals()['x_hash%s' % x]=np.sort(x2)
             print(globals()['x_hash%s' % x])            

[ 9224074046236128083  9227865158289475454  9229351712225673679 ...
 18443510138377475449 18445794649772851073 18446656124593965524]
[ 9224717234222054331  9227153475723306698  9231285630047250339 ...
 18443999699996328761 18445304129594335692 18446137253901635920]
[ 9227257094460995507  9227372289600986061  9229913402847975376 ...
 18444660049367318630 18445268674639339917 18445656393653657218]
[ 9225482563983374288  9225500275768108443  9229898577180112369 ...
 18444086560571320528 18445741535015244957 18446571519321713862]
[ 9224282938412692095  9224844555567814501  9225446168794521979 ...
 18445588323256274944 18446322422092191532 18446682228339522112]


In [ ]:
def find_duplicate(x1,xhash):
  hash1=convert_hash(x1)
  value=BinarySearch(xhash,hash1)
  if(value!=-1):
      mem=1
  elif(value==-1):
      mem=0
  return mem

In [ ]:
#check which model contain the example
def model_index_EO2(d1,d2,M):
  for i1 in range(M):
    traind=globals()['x_hash%s' % i1]
    #traind=shuffle(traind1)
    #traind=np.sort(traind)
    member=find_duplicate(d2,traind)
    if (member==1):
      index=i1
      break
  if (member==0):
    index=M
  return index

In [ ]:
stop!!

Exclusion Oracle-3: ImageHash

In [ ]:
def convert_Imhash(trainX1):
  lenth=len(trainX1)
  hashid=np.zeros(lenth)
  hash=[]
  for id in range(lenth):
    hashid[id]=id
    #covert in hashes
    im_array=Image.fromarray(np.uint8(trainX1[id])).convert('RGB')
    #im_array=Image.fromarray(trainX1[id].astype('uint8'), 'RGB')
    #im_array=trainX1[id]
    hash1 = imagehash.phash(im_array)
    #print(type(hash1))
    #hash1=str(hash1)
    #hash1=int(hash1,16)
    #print(type(hash1))
    #print(np.array(hash1))
    hash.append(hash1)
  return np.array(hash)

In [ ]:
def hamming_dist(h1,h2):
  n=len(h2)
  d=np.zeros(n)
  for i in range(n):
    d[i]=h1-h2[i]
  return d

In [ ]:
#check which model contain the example
def model_index_EO3(d1,d2,M):
  for i1 in range(M):
    traind=globals()['x_hash%s' % i1]
    #traind=shuffle(traind1)
    #traind=np.sort(traind)
    h1=convert_Imhash(d2)
    d=hamming_dist(h1,traind)
    member=np.count_nonzero(d < 14)
    #print(member)
    if (member>=1):
      index=i1
      break
  if (member==0):
    index=M
  return index

In [ ]:
#calculate and store hashes for M=5 #different model and their training dataset
#Only for imagehash
for x in range(0,M):
             #print(j,k+j)
             x1=globals()['x_traind%s' % x]
             x2=convert_Imhash(x1)
             globals()['x_hash%s' % x]=x2
             #print(globals()['x_hash%s' % x])
             

Exclusion Oracle-4:Classifier

In [ ]:
d_len=2500

In [ ]:
#PCA
from sklearn.decomposition import IncrementalPCA
from sklearn.decomposition import PCA
def get_PCA(xin,a):
    d0,d1,d2,d3=xin.shape
    xin=xin.reshape(d1*d0,d2*d3)
    xpca=xin
    pca = PCA(n_components=a) #a=number of components that we want to consider
    #pca=PCA(.5)
    fit = pca.fit(xpca)
    f=fit.components_
    return f

In [ ]:
def Overfit(model,x_test,Y_test):
  y_p=model.predict(x_test)
  y_p=y_p.reshape(num_class)
  #print(y_p)
  #print(Y_test)
  i3=np.argmax(Y_test)
  out=y_p[i3]
  #y_p=label(y_p)
  #print(y_p)
  y_t=label(Y_test)
  #print(y_t)
  #out=y_p[y_t]
  #out=accuracy_score(y_t,y_p)*100
  return out

In [ ]:
def Confidence_Vector(model,train,label):
  logit=model.predict(train)
  #print(logit)
  prob= special.softmax(logit, axis=1)
  prob=prob.reshape(num_class,1)
  out=prob[label]
  return out

In [ ]:
num2=d_len #subset training data from each model
if dataset=='Cifar10':
  d=np.concatenate((np.zeros(num2), np.ones(num2),np.ones(num2)*2,np.ones(num2)*3,np.ones(num2)*4))
elif dataset=='Cifar100':
  d=np.concatenate((np.zeros(num2), np.ones(num2),np.ones(num2)*2,np.ones(num2)*3))

d.shape

In [ ]:
def MIA_classM(nd,iter):
  mdt3=M #total Number of model
  mt3=nd #Total number of samples per model 
  m13=1  #number of different dataset participate
  datat3=np.zeros([mdt3*mt3*m13,M]) 
  datadt3=np.zeros([mdt3*mt3*m13,32*3*3])  #taking input image
  dataxt3=np.zeros([mdt3*mt3*m13,1]) 
  dataf=np.zeros([mdt3*mt3*m13,512]) #label
  ##create a dataset data1[samples,modelno,prediction_vectors]
#datax=[]
  k1=0
  for i in range(0,iter):
  #m=500
    x1=globals()['x_train_split%s' % i]
    y1=globals()['trainy_split%s' % i]
     #m=no. of samples, md=prediction of models , last=MNIST 10 prediction values
    for b1 in range(mt3):
       x2=x1[b1:b1+1]
       y2=y1[b1:b1+1]
    #print(x2.shape)
       ypred=np.zeros([mdt3])
       for b2 in range(mdt3):
         modelc=globals()['model_%s' % b2]
       #modelc.predict_proba(x2)
         yc=Confidence_Vector(modelc,x2,y2)
         ypred[b2]=yc
       #xd=x2.reshape(28*28)
     #Parallel(n_jobs=5, require='sharedmem')(delayed(indexM)(b2) for b2 in range(md))
       xpca=get_PCA(x2,3)
       d1,d2=xpca.shape
       xd=xpca.reshape(d1*d2)
       xr=y2
       #xf=to_feature_maps(x2)
       #xf=xf.reshape(512*1*1*1)
       dataxt3[k1,:]=xr
       datadt3[k1,:]=xd
       #dataf[k1,:]=xf
     #p=ypred[b2,:]
     #xp=[xr,p]
       #datak.append(xp)   #take the input vector as an array
       datat3[k1,:]=ypred
       k1=k1+1
  arrt3=np.hstack((datadt3,datat3,dataxt3))
  return arrt3


In [ ]:
arr1=MIA_classM(d_len,M)

In [ ]:
Y=d
X=arr1

In [ ]:
#split the dataset into train and test set
from sklearn.model_selection import train_test_split
X_train_ex, X_test_ex, y_train_ex, y_test_ex = train_test_split(X, Y, test_size=0.00001, random_state=42)

In [ ]:
#build a model for logistic Regression 
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
modelL = RandomForestClassifier(max_depth=20, random_state=42)
#modelL=svm.SVC()
#modelL = tree.DecisionTreeClassifier()
modelL.fit(X_train_ex,y_train_ex)

In [ ]:
#Threshold value
def predict_label(yp1,t):
  val=np.max(yp1)
  if (val>=t):
    lb=np.argmax(yp1)
  else:
    lb=M
  return lb

In [ ]:
def MIA_class(dataM,label,nd):
  iter=1
  mdt3=M #total Number of model
  mt3=nd #Total number of samples per model 
  m13=1  #number of different dataset participate
  datat3=np.zeros([mt3*m13,M]) 
  datadt3=np.zeros([mt3*m13,32*3*3])  #taking input image
  dataxt3=np.zeros([mt3*m13,1]) 
  dataf=np.zeros([mt3*m13,512]) #label
  ##create a dataset data1[samples,modelno,prediction_vectors]
#datax=[]
  k1=0
  for i in range(0,iter):
  #m=500
    x1=dataM#globals()['x_train_split%s' % i]
    y1=label#globals()['trainy_split%s' % i]
     #m=no. of samples, md=prediction of models , last=MNIST 10 prediction values
    for b1 in range(mt3):
       x2=x1[b1:b1+1]
       y2=y1[b1:b1+1]
    #print(x2.shape)
       ypred=np.zeros([mdt3])
       for b2 in range(mdt3):
         modelc=globals()['model_%s' % b2]
       #modelc.predict_proba(x2)
         yc=Confidence_Vector(modelc,x2,y2)
         ypred[b2]=yc
       #xd=x2.reshape(28*28)
     #Parallel(n_jobs=5, require='sharedmem')(delayed(indexM)(b2) for b2 in range(md))
       xpca=get_PCA(x2,3)
       d1,d2=xpca.shape
       xd=xpca.reshape(d1*d2)
       xr=y2
       #xf=to_feature_maps(x2)
       #xf=xf.reshape(512*1*1*1)
       dataxt3[k1,:]=xr
       datadt3[k1,:]=xd
       #dataf[k1,:]=xf
     #p=ypred[b2,:]
     #xp=[xr,p]
       #datak.append(xp)   #take the input vector as an array
       datat3[k1,:]=ypred
       k1=k1+1
  arrt3=np.hstack((datadt3,datat3,dataxt3))
  return arrt3


In [ ]:
def MIA_classNM(dataM,nd):
  mdt3=M #total Number of model
  mt3=nd #Total number of samples per model 
  m13=1  #number of different dataset participate
  datat3=np.zeros([mt3*m13,M]) 
  datadt3=np.zeros([mt3*m13,32*3*3])  #taking input image
  dataxt3=np.zeros([mt3*m13,1]) #label
  dataf=np.zeros([mdt3*mt3*m13,512])  #feature mapping
  ##create a dataset data1[samples,modelno,prediction_vectors]
#datax=[]
  k1=0
  for i in range(0,1):
  #m=500
    x1=dataM
    #y1=data_label
     #m=no. of samples, md=prediction of models , last=MNIST 10 prediction values
    for b1 in range(mt3):
       x2=x1[b1:b1+1]
       #y2=y1[b1:b1+1]
    #print(x2.shape)
       ypred=np.zeros([mdt3])
       for b2 in range(mdt3):
         modelc=globals()['model_%s' % b2]
       #modelc.predict_proba(x2)
         y2=modelc.predict(x2)
         y2=np.argmax(y2)
         yc=Confidence_Vector(modelc,x2,y2)
         ypred[b2]=yc
       #xd=x2.reshape(28*28)
     #Parallel(n_jobs=5, require='sharedmem')(delayed(indexM)(b2) for b2 in range(md))
       xpca=get_PCA(x2,3)
       d1,d2=xpca.shape
       xd=xpca.reshape(d1*d2)
       xr=y2
       #xf=to_feature_maps(x2)
       #xf=xf.reshape(512*1*1*1)
       dataxt3[k1,:]=xr
       datadt3[k1,:]=xd
       #dataf[k1,:]=xf
     #p=ypred[b2,:]
     #xp=[xr,p]
       #datak.append(xp)   #take the input vector as an array
       datat3[k1,:]=ypred
       k1=k1+1
  arrt3=np.hstack((datadt3,datat3,dataxt3))
  return arrt3

In [ ]:
arr2=MIA_class(test_data[0:5000],test_labels[0:5000],5000)

In [ ]:
arr=np.vstack((arr1,arr2))

In [ ]:
numt=5000#subset training data from each model
dt=(np.ones(numt)*M)
dt

In [ ]:
d=d.reshape(len(d),1)
dt=dt.reshape(len(dt),1)
d_total=np.vstack((d,dt))

In [ ]:
#final Train-Test set for thresholding
Xtest=arr
Ytest=d_total

In [ ]:
y_pred=modelL.predict_proba(Xtest)
#model_index=label(y_pred)
#accuracy_score(Ytest,y_pred)*100
#model_index=model_index.reshape(1000,6)
#model_index
y_pred

In [ ]:
th=np.linspace(0,1,20)
y_acc1=np.zeros(len(th))
for j in range(len(th)):
  ylab=np.zeros(len(y_pred))
  for i in range(len(y_pred)):
     ylab[i]=predict_label(y_pred[i,:],th[j])

  y_acc1[j]=accuracy_score(Ytest,ylab)*100

In [ ]:
from google.colab import files
test1 = plt.figure()
plt.plot(th,y_acc1,color="blue",marker='o',linewidth=2.0, linestyle="--",label='X_pca=288')
#plt.plot(th,y_acc,color="blue",marker='+',linewidth=1.5, linestyle="-",label='n_component=1')
plt.legend(fontsize=12)
plt.xlabel("Threshold (th)",fontsize=15)
plt.ylabel("Decision Accuracy",fontsize=15)
test1.show()
test1.set_facecolor('white')
test1.savefig('Acc_Vs_threshold_cifar10-5.pdf')
#files.download('Acc_Vs_threshold_cifar10-5.pdf')

In [ ]:
inx=np.argmax(y_acc1)
thres=th[inx+1]
thres

In [ ]:
y_acc1[inx+1]

In [ ]:
from scipy.spatial.distance import cdist
from sklearn.metrics.pairwise import cosine_similarity
def calcl_correlation():
  x1=get_PCA(im1,3)
  x1=x1.reshape(1,32*3*3)
  x2=get_PCA(im2,3)
  x2=x2.reshape(1,32*3*3)
  dist=cosine_similarity(x1,x2)
  return dist

In [ ]:
def model_index_EO4(d1,d2,M):
  arr_test=MIA_classNM(d1,len(d1))
  ypt=modelL.predict_proba(arr_test)
  indx=predict_label(ypt,thres)
  return indx

Exclusion Oracle-5: Chain

In [ ]:
def model_index_EO5(d1,d2,M,lab):
  index=model_index_EO2(d2,M)
  if index==M:
    index=model_index_EO3(d2,M)
    if index==M:
      index=model_index_EO4(d1,M)
    else:
      index=index
  else:
    index=index
  return index

transformation functions

In [ ]:
from scipy import ndimage
def rotate_image(x1,a):  #angle
  x2=np.zeros(x1.shape)
  x3=np.zeros(x1.shape)
  tp=x1.dtype
  x2=x2.astype(tp)
  x3=x3.astype(tp)
  for i in range(len(x1)):
    out1=ndimage.rotate(x1[i],a,reshape=False)
    #out2=ndimage.rotate(x1[i],-a,reshape=False)
    x2[i]=out1
    #x3[i]=out2
  return x2

calculate the EO accuracy

In [ ]:
dn=5000
train1=x_train_split0[0:dn]
trainx1=x_traind0[0:dn]
test1=test_data[5000:5000+dn]
testx1=testX1[5000:5000+dn]
r=10

In [ ]:
XT=x_train_split0[0:dn]
XT1=x_traind0[0:dn]
TestX=test_data[5000:10000]
TestX1=testX1[5000:10000]

In [ ]:
#rotation
train_data_rotate=rotate_image(XT,r)
trainX_data_rotate=rotate_image(XT1,r)
test_data_rotate=rotate_image(TestX,r)
testX_data_rotate=rotate_image(TestX1,r)

start evaluating PD attacks

In [ ]:
#define input
train_d=train1
train_x=trainx1
test_d=test1
test_x=testx1
yt=trainy_split0[0:dn]
ys=test_labels[5000:5000+dn]

In [ ]:
#calculate parameters from our technique: Only Model index for EO accuracy
#from joblib import Parallel, delayed
import time
def calc_EO_acc(data1,data2):
  if EO=='EO2':
    model_index=model_index_EO2
  elif EO=='EO3':
    model_index=model_index_EO3
  elif EO=='EO4':
    model_index=model_index_EO4
  elif EO=='EO5':
    model_index=model_index_EO5
  
  n=len(data1)
  logits=np.zeros((n,num_class))
  prob=np.zeros((n,num_class))
  labels=np.zeros(n)
  index_resulted=np.zeros(n)
  ts=.02
  for p1 in range(n):
    kk=0
    d1=data1[p1:p1+1]
    d2=data2[p1:p1+1]
    #sum1=np.zeros(10)
    sum2=np.zeros(num_class)
    index=model_index(d1,d2,M)
    index_resulted[p1]=index
  return index_resulted

In [ ]:
#calculate parameters from our combined
#from joblib import Parallel, delayed
import time
import timeit
def calc_attack_matrix(data1,data2):
  n=len(data1)
  logits=np.zeros((n,num_class))
  prob=np.zeros((n,num_class))
  labels=np.zeros(n)
  index_resulted=np.zeros(n)
  index_arr=np.zeros((n,293))
  ts=.02
  kk=0
  #Define Model Index
  if EO=='EO2':
    model_index=model_index_EO2
  elif EO=='EO3':
    model_index=model_index_EO3
  elif EO=='EO4':
    model_index=model_index_EO4
  elif EO=='EO5':
    model_index=model_index_EO5
  for p1 in range(n):
    d1=data1[p1:p1+1]
    d2=data2[p1:p1+1]
    #sum1=np.zeros(10)
    sum2=np.zeros(num_class)
    index=model_index(d1,d2,M)
    #print(arr_t.shape)
    #index_arr[p1,:]=arr_t
    index_resulted[p1]=index
    #print(index)
    for p2 in range(M):
      if(index!=p2):
        start = timeit.default_timer()
        modelh=globals()['model_%s' % p2]
        prob_d = modelh.predict(d1)
        #prob_d = special.softmax(logits_d)
        kk=kk+1
        #sum1=logits_d+sum1
        prob_vote=confidence(prob_d)
        sum2=prob_d+sum2
        #print(sum1)
        #print(sum2)
        stop = timeit.default_timer()
        ts=stop-start
      else:
        time.sleep(ts)
    #logits[p1,:]=sum1/100
    #print(logits)
    prob[p1,:]=sum2/kk*2
    #labels[p1,:]=np.argmax(logits[p1,:])
  return prob,index_resulted

In [ ]:
import timeit
print('Predict on train...')
start1 = timeit.default_timer()
#logits_traind,prob_traind,train_index = calc_attack_matrix(train_data[0:5000],trainX1[0:5000])
prob_traind,train_index = calc_attack_matrix(train_d,train_x)
stop1 = timeit.default_timer()
print('time required for members',stop1-start1)
print('Predict on test...')
start2 = timeit.default_timer()
prob_testd,test_index = calc_attack_matrix(test_d,test_x)
stop2 = timeit.default_timer()
print('time required for non-members',stop2-start2)

In [ ]:
#target model index
in1=np.zeros(dn)
in2=M*np.ones(dn)
index_target=np.hstack((in1,in2))
index_target

array([0., 0., 0., ..., 5., 5., 5.])

In [ ]:
result_index=np.hstack((train_index,test_index))
print(result_index)
accuracy_score(index_target,result_index)

In [ ]:
(unique, counts) = np.unique(train_index, return_counts=True)
frequencies = np.asarray((unique, counts)).T
frequencies

Probability Dependent Attack

In [ ]:
prob_trainf=prob_traind
prob_testf=prob_testd
yt=trainy_split0[0:dn]
ys=test_labels[5000:5000+dn]
attack=1      #define attacks. if 1-Th, 2-LR, 3-MLP

In [ ]:
# define what variables our attacker should have access to
attack_input = AttackInputData(
  logits_train = prob_trainf,
  logits_test = prob_testf,
  #loss_train = loss_train,
  #loss_test = loss_test,
  labels_train =yt,
  labels_test =ys
)

In [ ]:
# how should the data be sliced
slicing_spec = SlicingSpec(
    entire_dataset = True,
    by_class = False,
    by_percentiles = False,
    by_classification_correctness = True)

In [ ]:
# define the type of attacker model that we want to use
if attack==1:
  attack_types = [
    AttackType.THRESHOLD_ATTACK
]
elif attack==2:
  attack_types = [
    AttackType.LOGISTIC_REGRESSION
]
elif attack==3:
  attack_types = [
    AttackType.MULTI_LAYERED_PERCEPTRON
]

In [ ]:
# run the attack
attacks_result = mia.run_attacks(attack_input=attack_input,
                                 slicing_spec=slicing_spec,
                                 attack_types=attack_types)

In [ ]:
# summary by data slice (the best performing attacks per slice are presented)
print(attacks_result.summary(by_slices=True))

In [ ]:
#calculate test accuracy/utility
from sklearn.metrics import accuracy_score
print("calculate model utility")
test_labeld_result=np.argmax(prob_testf,axis=1)
accuracy_score(ys,test_labeld_result)

In [ ]:
#calculate train accuracy/utility
print("Calculate Member's accuracy")
from sklearn.metrics import accuracy_score
train_labeld_result=np.argmax(prob_trainf,axis=1)
accuracy_score(yt,train_labeld_result)

GAP Attack

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from sklearn.metrics import accuracy_score

In [ ]:
in1=np.zeros(dn)
in2=np.ones(dn)
Attack_target=np.hstack((in2,in1))
Attack_target

In [ ]:
def Gap_attack(prob_train,prob_test,label_train,label_test):
  result1=np.argmax(prob_train,axis=1)
  result2=np.argmax(prob_test,axis=1)
  result_label=np.hstack((result1,result2))
  target_label=np.hstack((label_train,label_test))
  target_label=np.array(target_label)
  g1=len(target_label)
  target_label=target_label.reshape(g1,1)
  result_label=np.array(result_label)
  result_label=result_label.reshape(g1,1)
  target_label=target_label.astype(int)
  result_label=result_label.astype(int)
  attack_result=np.equal(target_label, result_label)
  attack_result=attack_result.astype(int) 
  fpr, tpr, thresholds = metrics.roc_curve(Attack_target, attack_result, pos_label=1)
  AUC=metrics.auc(fpr, tpr)
  advantage=np.max(abs(fpr-tpr))
  return AUC, advantage

In [ ]:
#gap attack for non-private data
auc,adv=Gap_attack(prob_trainf,prob_testf,yt,ys)
print(auc)
print(adv)

In [ ]:
stop!!!

Label-Only Attack:Initialization

In [ ]:
from scipy import ndimage, misc
import matplotlib.pyplot as plt

In [ ]:
#initialize the flags
r=15   #1-15
target_model=model_np 
private_model=model_np#the model that needs to be evaluated
train=train_data[5000:10000]
test=test_data[0:5000]

In [ ]:
def make_fc():
  """Return conv_model as tf.keras.Model, without Softmax layer (only logits).
  To perform
  :param input_shape:
  :param depth:
  :param regularization:
  :param reg_constant:
  :param num_classes:
  :return:
  """
  fc_model = tf.keras.models.Sequential()
  #fc_model.add(tf.keras.layers.Dense(num_classes, kernel_regularizer=k_reg))
  fc_model.add(tf.keras.layers.Dense(10, activation=tf.keras.layers.ReLU(
        negative_slope=1e-2), kernel_initializer='glorot_normal'))
  fc_model.add(keras.layers.Dense(10, activation=tf.keras.layers.ReLU(
        negative_slope=1e-2), kernel_initializer='glorot_normal'))
  fc_model.add(tf.keras.layers.Dense(2, kernel_initializer='glorot_normal'))
  fc_model.add(tf.keras.layers.Softmax())
  return fc_model

RA attack

In [ ]:
def rotate_image(x1,a):  #angle
  x2=np.zeros(x1.shape)
  x3=np.zeros(x1.shape)
  tp=x1.dtype
  x2=x2.astype(tp)
  x3=x3.astype(tp)
  for i in range(len(x1)):
    out1=ndimage.rotate(x1[i],a,reshape=False)
    out2=ndimage.rotate(x1[i],-a,reshape=False)
    x2[i]=out1
    x3[i]=out2
  return x2,x3

In [ ]:
train_data_rotate1,train_data_rotate2=rotate_image(train_data[5000:10000],r)
test_data_rotate1,test_data_rotate2=rotate_image(test_data[0:5000],r)

In [ ]:
def calc_b(target_model,data,t):
  n=len(data)
  ytr=target_model.predict(data)
  ytr=np.argmax(ytr,axis=1)
  ytr=ytr.reshape(n,1)
  btr=(ytr-t)
  btr=np.where(btr==0,1,0)
  return btr,ytr

In [ ]:
t=train_labels[5000:10000].reshape(5000,1)
s=test_labels[0:5000].reshape(5000,1)
T=train_data[5000:10000].reshape(5000,32*32*3)
S=test_data[0:5000].reshape(5000,32*32*3)

In [ ]:
#calculate b
btr1,ytr1=calc_b(target_model,train_data_rotate1,t)
btr2,ytr2=calc_b(target_model,train_data_rotate2,t)
btr3,ytr3=calc_b(target_model,train,t)


bsr1,ysr1=calc_b(target_model,test_data_rotate1,s)
bsr2,ysr2=calc_b(target_model,test_data_rotate2,s)
bsr3,ysr3=calc_b(target_model,test,s)

out1=np.ones(5000)
out2=np.zeros(5000)
label1=np.hstack((out1,out2))
print(label1.shape)
#rearrange b
btr=np.hstack((btr3,btr1,btr2))
#btr=np.transpose(btr)
#print(btr.shape)
bsr=np.hstack((bsr3,bsr1,bsr2))
#bsr=np.transpose(bsr)
b=np.vstack((btr,bsr))
b.shape

In [ ]:
total=np.concatenate([t, s])
Bt=np.hstack((b,total))
Bt.shape
label=label1.flatten()
#label=to_categorical(label1)

In [ ]:
from sklearn.model_selection import train_test_split
t_data,s_data,t_label,s_label=train_test_split(Bt, label, test_size=0.1, random_state=42)

In [ ]:
#train a model
modelLB=make_fc()
optimizer = tf.keras.optimizers.Adam(learning_rate=.001)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# compile the model
modelLB.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
# train the model
history = modelLB.fit(t_data, t_label,
                       validation_data=(s_data, s_label),
                       batch_size=32, 
                       epochs=30)

In [ ]:
from sklearn.metrics import accuracy_score
mem=modelLB.predict(s_data)
mem=np.argmax(mem,axis=1)
#s_label_target=np.argmax(s_label,axis=1)
accuracy_score(mem,s_label)

In [ ]:
#Auc Score
from sklearn.metrics import roc_auc_score
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(mem, s_label, pos_label=1)
metrics.auc(fpr, tpr)

In [ ]:
np.max(abs(fpr-tpr))

In [ ]:
#calculate test accuracy/utility
from sklearn.metrics import accuracy_score
#test_labeld_result=np.argmax(prob_test,axis=1)
accuracy_score(t,ytr1)

In [ ]:
#calculate test accuracy/utility
from sklearn.metrics import accuracy_score
#test_labeld_result=np.argmax(prob_test,axis=1)
accuracy_score(s,ysr1)

In [ ]:
stop!!

Private Model

In [ ]:
def calc_b(private_model,data1,data2,t):    #if only model_stacking
  n=len(data1)
  ytr,indx=calc_attack_matrix(data1,data2)
  ytr=np.argmax(ytr,axis=1)
  ytr=ytr.reshape(n,1)
  btr=(ytr-t)
  btr=np.where(btr==0,1,0)
  return btr,ytr

In [ ]:
#create training and testing rotate data
train_data_rotateT1,train_data_rotateT2=rotate_image(train1,r)
train_X1_rotateT1,train_X1_rotateT2=rotate_image(trainx1,r)
test_data_rotateT1,test_data_rotateT2=rotate_image(test1,r)
test_X1_rotateT1,test_X1_rotateT2=rotate_image(testx1,r)

In [ ]:
t=yt.reshape(dn,1)
s=ys.reshape(dn,1)
T=train1.reshape(dn,32*32*3)
S=test1.reshape(dn,32*32*3)

In [ ]:
#calculate b
Btr1,Ytr1=calc_b(private_model,train_data_rotateT1,train_X1_rotateT1,t)
Btr2,Ytr2=calc_b(private_model,train_data_rotateT2,train_X1_rotateT2,t)
Btr3,Ytr3=calc_b(private_model,train1,trainx1,t)

Bsr1,Ysr1=calc_b(private_model,test_data_rotateT1,test_X1_rotateT1,s)
Bsr2,Ysr2=calc_b(private_model,test_data_rotateT2,test_X1_rotateT2,s)
Bsr3,Ysr3=calc_b(private_model,test1,testx1,s)

out1=np.ones(dn)
out2=np.zeros(dn)
label1=np.hstack((out1,out2))
print(label1.shape)
#rearrange b
Btr=np.hstack((Btr3,Btr1,Btr2))
#btr=np.transpose(btr)
#print(btr.shape)
Bsr=np.hstack((Bsr3,Bsr1,Bsr2))
#bsr=np.transpose(bsr)
B=np.vstack((Btr,Bsr))
B.shape

In [ ]:
total=np.concatenate([t, s])
Btotal=np.hstack((B,total))
Btotal.shape
label=label1.flatten()
#label=to_categorical(label1)

In [ ]:
from sklearn.metrics import accuracy_score
mem=modelLB.predict(Btotal)
mem=np.argmax(mem,axis=1)
#s_label_target=np.argmax(s_label,axis=1)
accuracy_score(mem,label)

In [ ]:
#Auc Score
from sklearn.metrics import roc_auc_score
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(mem, label, pos_label=1)
metrics.auc(fpr, tpr)

In [ ]:
np.max(abs(fpr-tpr))

In [ ]:
#calculate test accuracy/utility
from sklearn.metrics import accuracy_score
#test_labeld_result=np.argmax(prob_test,axis=1)
accuracy_score(t,Ytr1)

In [ ]:
#calculate test accuracy/utility
from sklearn.metrics import accuracy_score
#test_labeld_result=np.argmax(prob_test,axis=1)
accuracy_score(s,Ysr1)

In [ ]:
stop!!

TA attack

In [ ]:
#d=1
N=5
#d=np.array([0,1,-1,1,-1]) 
#d=np.array([0,3,2,1,-3,-2,-1,3,2,1,-3,-2,-1])  #d=3
d=np.array([0,5,4,3,2,1,-5,-4,-3,-2,-1,5,4,3,2,1,-5,-4,-3,-2,-1])
kv=np.ceil(len(d)/2)

In [ ]:
import random
def translate_images1(x1,d):  #d=distance
  x2=np.zeros(x1.shape)
  x3=np.zeros(x1.shape)
  tp=x1.dtype
  x2=x2.astype(tp)
  x3=x3.astype(tp)
  for i in range(len(x1)):
    test_x=x1[i]
    if d==0:
      i1=0
      j1=0
    else:
      i1=d#random.uniform(-d,d)
      j1=N-abs(i1)

    out1=ndimage.interpolation.shift(test_x,np.array([i1,j1,0,]))
    #print(i1,j1)
    #i2=0#random.uniform(-d,d)
    #j2=d
    #out2=ndimage.interpolation.shift(test_x,np.array([i2,j2,0]))
    
    #_____________________________________________#
    x2[i]=out1
    

  return x2

In [ ]:
import random
def translate_images2(x1,d):  #d=distance
  x2=np.zeros(x1.shape)
  x3=np.zeros(x1.shape)
  tp=x1.dtype
  x2=x2.astype(tp)
  x3=x3.astype(tp)
  for i in range(len(x1)):
    test_x=x1[i]
    j1=d#random.uniform(-d,d)
    i1=N-abs(j1)
    out1=ndimage.interpolation.shift(test_x,np.array([i1,j1,0]))
    #print(i1,j1)
    #i2=0#random.uniform(-d,d)
    #j2=d
    #out2=ndimage.interpolation.shift(test_x,np.array([i2,j2,0]))
    
    #_____________________________________________#
    x2[i]=out1
    

  return x2

In [ ]:
def calc_b(target_model,data,t,d,k):
  n=len(data)
  if k>=kv:
    translate_images=translate_images2
  else:
    translate_images=translate_images1
  datat=translate_images(data,d)
  ytr=target_model.predict(datat)
  ytr=np.argmax(ytr,axis=1)
  ytr=ytr.reshape(n,1)
  btr=(ytr-t)
  btr=np.where(btr==0,1,0)
  ytr=ytr.reshape(len(ytr))
  btr=btr.reshape(len(btr))
  return btr,ytr

In [ ]:
t=train_labels[5000:10000].reshape(5000,1)
s=test_labels[0:5000].reshape(5000,1)
T=train_data[5000:10000].reshape(5000,32*32*3)
S=test_data[0:5000].reshape(5000,32*32*3)

In [ ]:
btr=np.zeros((len(t),len(d)))
ytr=np.zeros((len(t),len(d)))
#indxt=np.zeros(len(t),len(d))
print('Calculate features for training data')
for i in range(len(d)):
  btr[:,i],ytr[:,i]=calc_b(target_model,train_data[5000:10000],t,d[i],i)

print('Calculate features for testing data')
bsr=np.zeros((len(t),len(d)))
ysr=np.zeros((len(t),len(d)))
#indxt=np.zeros(len(t),len(d))
for i in range(len(d)):
  bsr[:,i],ysr[:,i]=calc_b(target_model,test_data[0:5000],s,d[i],i)


print('rearrange b')
b=np.vstack((btr,bsr))
b.shape

In [ ]:
total=np.concatenate([t, s])
B=np.hstack((b,total))
B.shape

In [ ]:
out1=np.ones(dn)
out2=np.zeros(dn)
label1=np.hstack((out1,out2))
print(label1.shape)
label=label1.flatten()
#label=to_categorical(label1)

(10000,)


In [ ]:
from sklearn.model_selection import train_test_split
t_data,s_data,t_label,s_label=train_test_split(B, label, test_size=0.2, random_state=42)

In [ ]:
#train a model
modelLB=make_fc()
optimizer = tf.keras.optimizers.Adam(learning_rate=.01)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# compile the model
modelLB.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
# train the model
history = modelLB.fit(t_data, t_label,
                       validation_data=(s_data, s_label),
                       batch_size=32, 
                       epochs=30)

In [ ]:
from sklearn.metrics import accuracy_score
mem=modelLB.predict(s_data)
mem=np.argmax(mem,axis=1)
#s_label_target=np.argmax(s_label,axis=1)
accuracy_score(mem,s_label)

In [ ]:
#Auc Score
from sklearn.metrics import roc_auc_score
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(mem, s_label, pos_label=1)
metrics.auc(fpr, tpr)

In [ ]:
np.max(abs(fpr-tpr))

In [ ]:
#calculate test accuracy/utility
from sklearn.metrics import accuracy_score
#test_labeld_result=np.argmax(prob_test,axis=1)
#acct=accuracy_score(t,ytr1)+accuracy_score(t,ytr2)+accuracy_score(t,ytr3)+accuracy_score(t,ytr4)+accuracy_score(t,ytr5)
#print(acct/5)
acct=np.zeros(len(d))
for i in range(len(d)):
  acct[i]=accuracy_score(t,ytr[:,i])

print(np.average(acct))

0.8135999999999999


In [ ]:
#calculate test accuracy/utility
from sklearn.metrics import accuracy_score
#test_labeld_result=np.argmax(prob_test,axis=1)
#accs=accuracy_score(s,ysr1)+accuracy_score(s,ysr1)+accuracy_score(s,ysr1)+accuracy_score(s,ysr1)+accuracy_score(s,ysr1)
#print(accs/5)
accs=np.zeros(len(d))
for i in range(len(d)):
  accs[i]=accuracy_score(s,ysr[:,i])

print(np.average(accs))

0.659


In [ ]:
stop!!

Private Settings

In [ ]:
def calc_B(target_model,data1,data2,t,d,k):
  n=len(data1)
  if k>=kv:
    translate_images=translate_images2
  else:
    translate_images=translate_images1
  datat1=translate_images(data1,d)
  datat2=translate_images(data2,d)
  ytr,indx=calc_attack_matrix(datat1,datat2)
  ytr=np.argmax(ytr,axis=1)
  ytr=ytr.reshape(n,1)
  btr=(ytr-t)
  btr=np.where(btr==0,1,0)
  ytr=ytr.reshape(len(ytr))
  btr=btr.reshape(len(btr))
  indx=indx.reshape(len(indx))
  return btr,ytr,indx

In [ ]:
t=yt.reshape(dn,1)
s=ys.reshape(dn,1)
T=train1.reshape(dn,32*32*3)
S=test1.reshape(dn,32*32*3)

In [ ]:
btr=np.zeros((len(t),len(d)))
ytr=np.zeros((len(t),len(d)))
indxt=np.zeros((len(t),len(d)))
print('Calculate features for training data')
for i in range(len(d)):
  btr[:,i],ytr[:,i],indxt[:,i]=calc_B(target_model,train1,trainx1,t,d[i],i)

print('Calculate features for testing data')
bsr=np.zeros((len(t),len(d)))
ysr=np.zeros((len(t),len(d)))
indxs=np.zeros((len(t),len(d)))
for i in range(len(d)):
  bsr[:,i],ysr[:,i],indxs[:,i]=calc_B(target_model,test1,testx1,s,d[i],i)


print('rearrange b')
b1=np.vstack((btr,bsr))
b1.shape

In [ ]:
total=np.concatenate([t, s])
Btotal=np.hstack((b1,total))
Btotal.shape

(5000, 6)

In [ ]:
out1=np.ones(dn)
out2=np.zeros(dn)
label1=np.hstack((out1,out2))
print(label1.shape)
label=label1.flatten()
#label=to_categorical(label1)

(5000,)


In [ ]:
from sklearn.metrics import accuracy_score
mem=modelLB.predict(Btotal)
mem=np.argmax(mem,axis=1)
#s_label_target=np.argmax(s_label,axis=1)
accuracy_score(mem,label)

0.5114

In [ ]:
#Auc Score
from sklearn.metrics import roc_auc_score
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(mem, label, pos_label=1)
metrics.auc(fpr, tpr)

0.5134056807348233

In [ ]:
#calculate test accuracy/utility
from sklearn.metrics import accuracy_score
#test_labeld_result=np.argmax(prob_test,axis=1)
#acct=accuracy_score(t,ytr1)+accuracy_score(t,ytr2)+accuracy_score(t,ytr3)+accuracy_score(t,ytr4)+accuracy_score(t,ytr5)
#print(acct/5)
acct=np.zeros(len(d))
for i in range(len(d)):
  acct[i]=accuracy_score(t,ytr[:,i])

print(np.average(acct))

0.69104


In [ ]:
#calculate test accuracy/utility
from sklearn.metrics import accuracy_score
#test_labeld_result=np.argmax(prob_test,axis=1)
#acct=accuracy_score(t,ytr1)+accuracy_score(t,ytr2)+accuracy_score(t,ytr3)+accuracy_score(t,ytr4)+accuracy_score(t,ytr5)
#print(acct/5)
accs=np.zeros(len(d))
for i in range(len(d)):
  accs[i]=accuracy_score(s,ysr[:,i])

print(np.average(accs))

0.6668


In [ ]:
np.max(abs(fpr-tpr))

0.026811361469646466

In [ ]:
#calculate test accuracy/utility
from sklearn.metrics import accuracy_score
#test_labeld_result=np.argmax(prob_test,axis=1)
#acct=accuracy_score(t,ytr1)+accuracy_score(t,ytr2)+accuracy_score(t,ytr3)+accuracy_score(t,ytr4)+accuracy_score(t,ytr5)
#print(acct/5)
accindx=np.zeros(len(d))
for i in range(len(d)):
  result_index=np.hstack((indxt[:,i],indxs[:,i]))
  accindx[i]=accuracy_score(index_target,result_index)

print(np.average(accindx))

In [ ]:
stop!!

Boundary Distance Attack

In [ ]:
n1=5000

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_curve

In [ ]:
def continuous_rand_robust(model, dsx,dsy, max_samples=100, noise_samples=250, stddev=0.025, input_dim=shape,
                           num=[1, 2, 3, 4, 5, 10, 20, 30, 40, 50, 75, 100, 150, 250]): #350, 500, 750, 1000, 1500, 2000, 2500]):
  """Calculate robustness to random noise for Adv-x MI attack on continuous-featureed datasets (+ UCI adult).
  :param model: model to approximate distances on (attack).
  :param ds: tf dataset should be either the training set or the test set.
  :param max_samples: maximum number of samples to take from the ds
  :param noise_samples: number of noised samples to take for each sample in the ds.
  :param stddev: the standard deviation to use for Gaussian noise (only for Adult, which has some continuous features)
  :param input_dim: dimension of inputs for the dataset.
  :param num: subnumber of samples to evaluate. max number is noise_samples
  :return: a list of lists. each sublist of the accuracy on up to $num noise_samples.
  """
  robust_accs = [[] for _ in num]
  iter=len(dsx)
  labels=dsy
  for i in range(iter):
    x=dsx[i:i+1]
    noise = stddev * np.random.randn(noise_samples, 32,32,3)
    x_noisy = np.clip(x + noise, 0, 1)
    #print(noise.shape)
    prob=model.predict(x_noisy)
    pred=np.argmax(prob,axis=1)
    #print(pred.shape)
    for idx, n in enumerate(num):
            if n == 0:
              robust_accs[idx].append(1)
            else:
              robust_accs[idx].append(np.mean(pred[:n] == labels[i]))
  return robust_accs

In [ ]:
def continuous_rand_robust_target(model, dsx,dsy,dX, max_samples=100, noise_samples=250, stddev=0.025, input_dim=shape,
                           num=[1, 2, 3, 4, 5, 10, 20,30, 40, 50, 75, 100, 150, 250]):#350, 500, 750, 1000, 1500, 2000, 2500]):
  """Calculate robustness to random noise for Adv-x MI attack on continuous-featureed datasets (+ UCI adult).
  :param model: model to approximate distances on (attack).
  :param ds: tf dataset should be either the training set or the test set.
  :param max_samples: maximum number of samples to take from the ds
  :param noise_samples: number of noised samples to take for each sample in the ds.
  :param stddev: the standard deviation to use for Gaussian noise (only for Adult, which has some continuous features)
  :param input_dim: dimension of inputs for the dataset.
  :param num: subnumber of samples to evaluate. max number is noise_samples
  :return: a list of lists. each sublist of the accuracy on up to $num noise_samples.
  """
  robust_accs = [[] for _ in num]
  iter=len(dsx)
  labels=dsy
  for i in range(iter):
    x=dsx[i:i+1]
    X=dX[i:i+1]
    noise = stddev * np.random.randn(noise_samples, 32,32,3)
    x_noisy = np.clip(x + noise, 0, 1)
    X_noisy=np.clip(X + noise, 0, 255)
    #print(noise.shape)
    print('error1')
    prob,indx=calc_attack_matrix(x_noisy,X_noisy)
    pred=np.argmax(prob,axis=1)
    print('error2')
    #print(pred)
    for idx, n in enumerate(num):
            if n == 0:
              robust_accs[idx].append(1)
            else:
              robust_accs[idx].append(np.mean(pred[:n] == labels[i]))
  return robust_accs

non-Private

In [ ]:
noise_source_in=continuous_rand_robust(target_model,train_data[0:n1],train_labels[0:n1])
noise_source_out=continuous_rand_robust(target_model,test_data[0:n1],test_labels[0:n1])

private

In [ ]:
import timeit
noise_target_in=continuous_rand_robust_target(model_np,train_data[n1:n1*2],train_labels[n1:n1*2],trainX1[n1:n1*2])
noise_target_out=continuous_rand_robust_target(model_np,test_data[n1:n1*2],test_labels[n1:n1*2],testX1[n1:n1*2])

In [ ]:
source_m = np.concatenate([np.ones(n1),np.zeros(n1)], axis=0)
target_m = np.concatenate([np.ones(n1),np.zeros(n1)], axis=0)

In [ ]:
def get_max_accuracy(y_true, probs, thresholds=None):
  """Return the max accuracy possible given the correct labels and guesses. Will try all thresholds unless passed.
  Args:
    y_true: True label of `in' or `out' (member or non-member, 1/0)
    probs: The scalar to threshold
    thresholds: In a blackbox setup with a shadow/source model, the threshold obtained by the source model can be passed
      here for attackin the target model. This threshold will then be used.
  Returns: max accuracy possible, accuracy at the threshold passed (if one was passed), the max precision possible,
   and the precision at the threshold passed.
  """
  attack_adv=[]
  if thresholds is None:
    fpr, tpr, thresholds = roc_curve(y_true, probs)
    attack_adv.append(abs(tpr-fpr))
    

  accuracy_scores = []
  precision_scores = []
  for thresh in thresholds:
    accuracy_scores.append(accuracy_score(y_true,
                                          [1 if m > thresh else 0 for m in probs]))
    precision_scores.append(precision_score(y_true, [1 if m > thresh else 0 for m in probs]))

  accuracies = np.array(accuracy_scores)
  precisions = np.array(precision_scores)
  attack_adv=np.array(attack_adv)
  max_accuracy = accuracies.max()
  max_precision = precisions.max()
  #print(type(attack_adv))
  #print(attack_adv)
  if (attack_adv.size !=0):
      max_adv=np.max(attack_adv)
      print(max_adv)
  max_accuracy_threshold = thresholds[accuracies.argmax()]
  max_precision_threshold = thresholds[precisions.argmax()]
  return max_accuracy, max_accuracy_threshold, max_precision, max_precision_threshold

In [ ]:
def get_threshold(source_m, source_stats, target_m, target_stats):
  """ Train a threshold attack model and get teh accuracy on source and target models.
  Args:
    source_m: membership labels for source dataset (1 for member, 0 for non-member)
    source_stats: scalar values to threshold (attack features) for source dataset
    target_m: membership labels for target dataset (1 for member, 0 for non-member)
    target_stats: scalar values to threshold (attack features) for target dataset
  Returns: best acc from source thresh, precision @ same threshold, threshold for best acc,
    precision at the best threshold for precision. all tuned on source model.
  """
  # find best threshold on source data
  acc_source, t, prec_source, tprec = get_max_accuracy(source_m, source_stats)

  # find best accuracy on test data (just to check how much we overfit)
  acc_test, _, prec_test, _ = get_max_accuracy(target_m, target_stats)

  # get the test accuracy at the threshold selected on the source data
  acc_test_t, _, _, _ = get_max_accuracy(target_m, target_stats, thresholds=[t])
  _, _, prec_test_t, _ = get_max_accuracy(target_m, target_stats, thresholds=[tprec])
  print("acc src: {}, acc test (best thresh): {}, acc test (src thresh): {}, thresh: {}".format(acc_source, acc_test,
                                                                                                acc_test_t, t))
  print(
    "prec src: {}, prec test (best thresh): {}, prec test (src thresh): {}, thresh: {}".format(prec_source, prec_test,
                                                                                               prec_test_t, tprec))

  return acc_test_t, prec_test_t, t, tprec

In [ ]:
for i in range(len(noise_source_in)):
            noise_source = np.concatenate([noise_source_in[i], noise_source_out[i]], axis=0)
            noise_target = np.concatenate([noise_target_in[i], noise_target_out[i]], axis=0)
            get_threshold(source_m, noise_source, target_m, noise_target)